In [3]:
import eleanor
import os
import numpy as np
import glob
from tess_sip import SIP
from astropy.coordinates import SkyCoord
from astropy import units as u
import lightkurve as lk
import eleanor
from astroquery.mast import Tesscut
from eleanor.utils import SearchError
import tess_cpm
import sys
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import warnings
from scipy.signal import find_peaks
from scipy.optimize import curve_fit
from scipy.ndimage import gaussian_filter1d

sys.path.insert(0, "../")

import warnings
warnings.filterwarnings('ignore')

from michael import janet

In [4]:
uni = pd.read_csv("/Users/oliver hall/Research/unicorn/data/unicorn.csv", index_col=0)

index = uni[uni.source_id == int(48977371798954368)].index[0]

In [6]:
j = janet.boot(uni, index=index)


In [10]:
coords = SkyCoord(ra = j.ra, dec = j.dec, unit = (u.deg, u.deg))


In [12]:
Tesscut.get_sectors(coordinates=coords)

sectorName,sector,camera,ccd
str14,int64,int64,int64
tess-s0043-3-3,43,3,3
tess-s0044-2-4,44,2,4


In [13]:
Tesscut.download_cutouts(coordinates=coords, size = 50, path='/Users/oliver hall/.michael/tesscut/')

Inflating...


Local Path
str90
/Users/oliver hall/.michael/tesscut/tess-s0044-2-4_64.763496_19.544165_50x50_astrocut.fits
/Users/oliver hall/.michael/tesscut/tess-s0043-3-3_64.763496_19.544165_50x50_astrocut.fits
